# Section 3.7 — Inventory of statistical tests

This notebook contains the code examples from [Section 3.7 Inventory of statistical tests]() from the **No Bullshit Guide to Statistics**.

#### Notebook setup

In [1]:
# load Python modules
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Plot helper functions
from plot_helpers import plot_pdf
from plot_helpers import savefigure

In [3]:
# Figures setup
plt.clf()  # needed otherwise `sns.set_theme` doesn't work
from plot_helpers import RCPARAMS
RCPARAMS.update({'figure.figsize': (10, 3)})   # good for screen
# RCPARAMS.update({'figure.figsize': (5, 1.6)})  # good for print
sns.set_theme(
    context="paper",
    style="whitegrid",
    palette="colorblind",
    rc=RCPARAMS,
)

# Useful colors
snspal = sns.color_palette()
blue, orange, purple = snspal[0], snspal[1], snspal[4]

# High-resolution please
%config InlineBackend.figure_format = 'retina'

# Where to store figures
DESTDIR = "figures/stats/inventory"

<Figure size 640x480 with 0 Axes>

In [4]:
# set random seed for repeatability
np.random.seed(42)

In [5]:
#######################################################

## Definitions

## Assumptions

## NHST procedure

## Categorization of statistical test recipes

## Z-Tests

### One-sample $z$-test

See the [examples/one_sample_z-test.ipynb](../examples/one_sample_z-test.ipynb) notebook.

## Proportion tests
### One-sample $z$-test for proportions
### Binomial test
### Two-sample $z$-test for proportions

In [6]:
from statsmodels.stats.proportion import proportions_ztest

## T-tests
### One sample $t$-test

See the [examples/one_sample_t-test.ipynb](../examples/one_sample_t-test.ipynb) notebook.

### Welch's two-sample $t$-test

(explain pooled variance as special case "Two sample t-test", but inferior)

### Paired $t$-test

## Chi-square tests

### Chi-square test for goodness of fit

### Chi-square test of independence

### Chi-square test for homogeneity

### Chi-square test for the population variance

## Analysis of variance (ANOVA) tests

### One-way analysis of variance (ANOVA)

### Two-way ANOVA

## Nonparametric tests
Use when assumptions for other tests not valid

### Sign test for the population median

### One-sample Wilcoxon signed-rank test

### Wilcoxon rank sum test

### Kruskal-Wallis analysis of variance by ranks

## Resampling methods

### Simulation tests

### Two-sample permutation test

### Permutation ANOVA

## Miscellaneous tests

### Equivalence tests

### Kolmogorov-Smirnov test

### Shapiro-Wilk normality test

## Explanations

## Discussion

## Exercises

## Links